In [2]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv()

True

ChatModels are instances of LangChain Runnables - which means they expose a standard interface for interacting with them.

**What are Chat Models**
- Use a sequence of messages as inputs and return chat messages as outputs as opposed to plain text.
- These are newer models, in contrast to the older LLM models.
- These models support the assignment of distinct roles to conversation messages, helping to differentiate from AI, users and instructions such as system messages for example > HumanMessage, SystemMessage (instructions and config), AIMessage (output from a model)

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

In [5]:
messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d33be0b6-d14b-445a-8272-e301d58a56ea-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

# OutputParsers

The response from the model is an AIMessage. This is an object and it's properties include content, usage_metadata etc. Oftentimes we only want to work with the string response that is stored in the **contents** property. This is where parsers come in useful.

In [6]:
from langchain_core.output_parsers import StrOutputParser

In [7]:
# Instantiate the parser
parser = StrOutputParser()

In [10]:
# Store the AIMessage response from the language model in a variable
result = model.invoke(messages)

# The parser accepts a BaseMessage as an argument. AIMessage is a subclass of the BaseMessage
parser.invoke(result)

'Ciao!'

### Chain

A more common way to do the above is by chaining the model with the output parser.
- The chain will take on the input type of the language model --> string or list of messages
- The chain outputs the type of the output parser --> string

In [11]:
# Define the chain
chain = model | parser

In [12]:
# Invoke the chain
chain.invoke(messages)

'Ciao!'

# Prompt Templates

Flow:
Raw user input --> Prompt --> Language model

In [13]:
from langchain_core.prompts import ChatPromptTemplate

In [14]:
# System instructions - context for the model. Not seen by the user in the chat
system_template = "Translate the following into {language}"

In [15]:
# Instantiate a prompt template from a list of message templates.
# The messages can take on a variety of formats
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), 
     ("user", "{text}")]
)

In [16]:
# Invoke the runnable
result = prompt_template.invoke({"language": "italian", "text": "How are you?"})

In [18]:
# The result of invoking the prompt template is a ChatPromptValue object
# This contains the sequence of messages but with the correct BaseMessage subclasses
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian'), HumanMessage(content='How are you?')])

In [21]:
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian'), HumanMessage(content='How are you?')])

# Chaining together components with LCEL

In [22]:
chain = prompt_template | model | parser

In [23]:
# Now that the prompt_template is at the start of the chain, the input to the chain is a dictionary
chain.invoke({"language": "spanish", "text": "What are you doing today"})

'¿Qué estás haciendo hoy?'

In [25]:
user_language = input("What language do you want to translate into: ")

In [26]:
user_text = input("What phrase would you like translated: ")

In [27]:
chain.invoke({"language": user_language, 
              "text": user_text})

'Du bist ein Hund!'